# Step 1 : Identify training neighbors

In [1]:
model_params = {
    "max_atoms" : int(60), "num_atom_features" : int(62), "max_degree" : int(5), "num_bond_features" : int(6),
    "graph_conv_width" : [128,128,128], "conv1d_filters" : int(128), "conv1d_size" : int(29), "dropout_encoder" : 0.25,
    "conv1d_filters_dist" : [128,128], "conv1d_size_dist" : [17,1], "dropout_dist" : 0.25, "pool_size" : int(4),
    "dense_size" : [256,128,128], "l2reg" : 0.01, "dist_thresh" : 0.2, "lr" : 0.001 ,"ConGauss":False
}

The model_params dictionary contains the parameters to build the deepSIBA siamese GCN architecture, more specifically:
1. **max_atoms, num_atom_features, max_degree and num_bond_features** refer to the parameters needed to featurize the input chemical structures. For more information, refer to the *ESI of the deepSIBA publication*.
2. **graph_conv_width, conv1d_filters, conv1d_size, dropout_encoder** refer to the parameters of the siamese graph encoders.
3. **conv1d_filters_dist, conv1d_size_dist, dropout_dist, pool_size, dense_size, l2reg** refer to the parameters of the distance module.
4. **dist_thresh** is the distance threshold to consider 2 chemical structures similar in biological effect (needed for custom training metrics).
5. **lr** is the learning rate.
6. **ConGauss** is by default set to False. **Set it True, only if training becomes difficult due to Loss becoming frequently Inf.** If set to True, a Gaussian Layer constrained to 0 to 1 is used instead of the original Gaussian layer.

In [2]:
inference_params = {
    "query_smile" : "CCCC1=CC2=C(C=CC(O2)(C)CCC=C(C)C)C(=C1C(=O)O)O", 
    "cell_line" : "a375", "split" : "alldata" ,
    "output_dir" : "C:/Users/user/Documents/deepSIBA/results/inference_test_jan_4" , "model_path" : "", 
    "atom_limit" : int(60), "N_models" : int(50),
    "name_pattern" : "siam_no_augment", "fold_id" : int(0),
    "neighbor_threshold" : 0.22
}

The inference_params dictionary contains the parameters required to identify the training neighbors to a query:

1. **query_smile** is the smile string of the chemical structure.
2. **cell_line** is the cellular model of choice out of **(a375,pc3,vcap,mcf7)** for which we have enough available data. Later a merged option will be added.
3. **split** is one of **(train_test_split,5_fold_cv_split,alldata,custom)**. The split selected defines the trained model ensemble that will be loaded. For the screening application the **alldata** split is suggested, where models are trained on the entirety of available data. If **custom** is selected the user must provide a path in **model_path** to load the custom trained models (up to models/ directory).
4. **output_dir** full path to the desired output directory to write results. The Chembl screening is performed in parts due to the size of the database.
5. **atom_limit** the specified model_params of the trained models, when the split is not **custom** these should be 60.
6. **N_models** is the number of total already trained models and at the same time the models included in the ensembled prediction.
7. **name_pattern** is the pattern of the name of files of models' saved weights. **For example** if the weights are saved in files with names such as **siam_no_augment_18.h5** the **pattern is siam_no_augment** .
8. **fold_id** is an integer, if split == 5_fold_cv_split the fold_id should be 0,1,2,3 or 4 (one less than the corresponding folder's name for this fold), in any other cases the fold_id is not used
9. **inference_threshold** only keep neighbors with predicted value below the threshold.

In [2]:
from deepSIBA_inference_predict import siba_training_neighbors

Using TensorFlow backend.


In [3]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

The following GPU devices are available: /device:GPU:0


In [ ]:
siba_training_neighbors(inference_params, model_params)

In [5]:
query_list = ["CCCCCC1=CC(=C2C=CC(OC2=C1)(C)CCC=C(C)C)O","CCCCCC1=CC2=C(C=CC(O2)(C)CCC=C(C)C)C(=C1C(=O)O)O",
             "CCCC1=CC(=C2C=CC(OC2=C1)(C)CCC=C(C)C)O","CCCC1=CC2=C(C=CC(O2)(C)CCC=C(C)C)C(=C1C(=O)O)O",
             "CCCCCC1=CC(=C2C3C4C(C3(C)C)CCC4(OC2=C1)C)O","CCCCCC1=CC2=C(C3C4C(C3(C)C)CCC4(O2)C)C(=C1C(=O)O)O",
             "CCCCCC1=CC(=C(C(=C1)O)C2C=C(CCC2C(=C)C)C)O","CCCCCC1=CC(=C(C(=C1)OC)C2C=C(CCC2C(=C)C)C)O",
             "CCCCCC1=CC(=C(C(=C1C(=O)O)O)C2C=C(CCC2C(=C)C)C)O","CC1=CC(C(CC1)C(=C)C)C2=C(C=C(C=C2O)C)O",
             "CC1=C[C@@H](C2=C(O)C=C(CCCC)C=C2O)[C@H](C(C)=C)CC1","CCCC1=CC(=C(C(=C1)O)C2C=C(CCC2C(=C)C)C)O",
             "CCCC1=CC(=C(C(=C1C(=O)O)O)C2C=C(CCC2C(=C)C)C)O","OC(C=C(C=C1O)CCCCCCC)=C1[C@H]2[C@H](C(C)=C)CCC(C)=C2",
             "OC(C=C(C=C1O)CCCCC)=C1[C@H]2[C@H](C(C)=C)C[C@H](O)C(C)=C2","CCCCCC1=CC(=C2C3C(CCC(C3OC2=C1C(=O)O)(C)O)C(=C)C)O",
             "CCCCCC1=CC(=C2C3C(CCC(C3OC2=C1)(C)O)C(=C)C)O","CCCCCC1=CC(=C(C(=C1)O)CC=C(C)CCC=C(C)C)O",
             "CC1=CC2=C(C(C)(C)OC3=C2C(OC)=CC(CCCCC)=C3)C=C1","CCCCCC1=CC(=C(C(=C1C(=O)O)O)CC=C(C)CCC=C(C)C)O",
             "CCCCCC1=CC(=C(C(=C1C(=O)O)O)CC=C(C)CCC=C(C)C)OC","CCCC1=CC(=C(C(=C1)O)CC=C(C)CCC=C(C)C)O",
             "CCCC1=CC(=C(C(=C1C(=O)O)O)CC=C(C)CCC=C(C)C)O","OC1=C(C/C=C(C)/CC/C=C(C)/C)C(O)=CC(C)=C1",
             "OC1=C(C/C=C(C)/CC/C=C(C)/C)C(O)=C(C(O)=O)C(C)=C1","CCCCCC1=CC(=C(C(=C1)O)C2=C(C=CC(=C2)C)C(=C)C)O",
             "CCCCCC1=CC(=C2C(=C1)OC(C3=C2C=C(C=C3)C)(C)C)O","CCCCCC1=CC2=C(C3=C(C=CC(=C3)C)C(O2)(C)C)C(=C1C(=O)O)O",
             "CCCC1=CC(=C2C(=C1)OC(C3=C2C=C(C=C3)C)(C)C)O","CCCCCC1=CC(=C2C(=C1)OC(C3=C2C(C(CC3)(C)O)O)(C)C)O",
             "CCCCCC1=CC(=C2C3CC(=CCC3C(OC2=C1)(C)C)C)O","CC1=CC[C@@]2([H])C(C)(C)OC(C=C(CCCCC)C(C(O)=O)=C3O)=C3[C@]2([H])C1",
             "CCCCCC1=CC(=C2C3C=C(CCC3C(OC2=C1)(C)C)C)O","CCCCC1=CC(=C2C3C=C(CCC3C(OC2=C1)(C)C)C)O",
             "CCCCCC1=CC2=C(C3C=C(CCC3C(O2)(C)C)C)C(=C1C(=O)O)O","CCCCC1=CC2=C([C@@H]3C=C(CC[C@H]3C(O2)(C)C)C)C(=C1)O",
             "CC1=CC2C(CC1)C(OC3=CC(=CC(=C23)O)C)(C)C","CC1=CC2C(CC1)C(OC3=C2C(=C(C(=C3)C)C(=O)O)O)(C)C",
             "CC1=CC2C(CC1)C(OC3=C(C(=CC(=C23)O)C)C(=O)O)(C)C","CCCC1=CC(=C2C3C=C(CCC3C(OC2=C1)(C)C)C)O",
             "CCCC1=CC2=C(C3C=C(CCC3C(O2)(C)C)C)C(=C1C(=O)O)O","CC1=C[C@]2([H])[C@@](C(C)(C)OC3=C2C(O)=CC(CCCCCCC)=C3)([H])CC1",
             "CCCCCC1=CC(=C2C(=C1)OC3=C(C=CC(=C23)C(C)C)C)O","CCCCCC1=CC(=C2C3C(CCC(C3O)(C)O)C(OC2=C1)(C)C)O",
             "CCCCCC1=CC2=C3C(OC(C)(C)C4C3CC(C)(CC4)O2)=C1","CCCCCC1=CC(=C2C(=C1)OC3=C(C=CC(=C23)C(=C)C)C)O",
             "CCCCCC1=CC(=C2C3C=C(CCC3C(OC2=C1)(C)C)C)O"]

In [6]:
for i in range(len(query_list)):
    inference_params = {
    "query_smile" : query_list[i], 
    "cell_line" : "npc", "split" : "alldata" ,
    "output_dir" : "cannabis_pathways_npc/inference_test_jan_%s"%i , "model_path" : "", 
    "atom_limit" : int(60), "N_models" : int(50),
    "name_pattern" : "model", "fold_id" : int(0),
    "neighbor_threshold" : 0.18
    }
    siba_training_neighbors(inference_params, model_params)

W1008 15:32:34.306509 140481856808768 deprecation_wrapper.py:119] From /home/biolab/miniconda3/envs/tf1/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1008 15:32:34.311409 140481856808768 deprecation_wrapper.py:119] From /home/biolab/miniconda3/envs/tf1/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1008 15:32:34.312103 140481856808768 deprecation_wrapper.py:119] From /home/biolab/miniconda3/envs/tf1/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W1008 15:32:34.425627 140481856808768 deprecation_wrapper.py:119] From /home/biolab/miniconda3/envs/tf1/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_defau

Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished predicting against the training set
Finished p